# Задание 7. Машинное обучение: предсказание погашения кредита

## Гипотеза
**Можно ли предсказать, будет ли кредит погашен, на основе характеристик заемщика?**

Мы предполагаем, что такие факторы как:
- Кредитный рейтинг (чем выше рейтинг, тем выше вероятность погашения)
- Годовой доход (чем выше доход, тем выше вероятность погашения)
- Коэффициент долг/доход (чем ниже коэффициент, тем выше вероятность погашения)
- История просрочек (чем меньше просрочек, тем выше вероятность погашения)
- Количество открытых счетов
- Процентная ставка

влияют на вероятность погашения кредита.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, mean_absolute_error, r2_score
from sklearn.preprocessing import StandardScaler, LabelEncoder
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 50)
pd.set_option('display.float_format', lambda x: '%.4f' % x)

print("Библиотеки загружены успешно!")

Библиотеки загружены успешно!


## 1. Загрузка и подготовка данных

In [36]:
# Загрузка данных из lab6
df = pd.read_csv('../lab6/data/loan_dataset_20000.csv')

# Переименование столбцов на русский язык
column_mapping = {
    'age': 'Возраст',
    'gender': 'Пол',
    'marital_status': 'Семейное_положение',
    'education_level': 'Уровень_образования',
    'annual_income': 'Годовой_доход',
    'monthly_income': 'Месячный_доход',
    'employment_status': 'Статус_занятости',
    'debt_to_income_ratio': 'Коэффициент_долг_доход',
    'credit_score': 'Кредитный_рейтинг',
    'loan_amount': 'Сумма_кредита',
    'loan_purpose': 'Цель_кредита',
    'interest_rate': 'Процентная_ставка',
    'loan_term': 'Срок_кредита',
    'installment': 'Ежемесячный_платеж',
    'grade_subgrade': 'Оценка_субкласс',
    'num_of_open_accounts': 'Количество_открытых_счетов',
    'total_credit_limit': 'Общий_кредитный_лимит',
    'current_balance': 'Текущий_баланс',
    'delinquency_history': 'История_просрочек',
    'public_records': 'Публичные_записи',
    'num_of_delinquencies': 'Количество_просрочек',
    'loan_paid_back': 'Кредит_погашен'
}

df = df.rename(columns=column_mapping)

print(f"Размерность датасета: {df.shape}")
print(f"\nПервые строки:")
df.head()

Размерность датасета: (20000, 22)

Первые строки:


,Возраст,Пол,Семейное_положение,Уровень_образования,Годовой_доход,Месячный_доход,Статус_занятости,Коэффициент_долг_доход,Кредитный_рейтинг,Сумма_кредита,Цель_кредита,Процентная_ставка,Срок_кредита,Ежемесячный_платеж,Оценка_субкласс,Количество_открытых_счетов,Общий_кредитный_лимит,Текущий_баланс,История_просрочек,Публичные_записи,Количество_просрочек,Кредит_погашен
0,59,Male,Married,Master's,24240.1900,2020.0200,Employed,0.0740,743,17173.7200,Car,13.3900,36,581.8800,B5,7,40833.4700,24302.0700,1,0,1,1
1,72,Female,Married,Bachelor's,20172.9800,1681.0800,Employed,0.2190,531,22663.8900,Debt consolidation,17.8100,60,573.1700,F1,5,27968.0100,10803.0100,1,0,3,1
2,49,Female,Single,High School,26181.8000,2181.8200,Employed,0.2340,779,3631.3600,Business,9.5300,60,76.3200,B4,2,15502.2500,4505.4400,0,0,0,1
3,35,Female,Single,High School,11873.8400,989.4900,Employed,0.2640,809,14939.2300,Other,7.9900,36,468.0700,A5,7,18157.7900,5525.6300,4,0,5,1
4,63,Other,Single,Other,25326.4400,2110.5400,Employed,0.2600,663,16551.7100,Car,15.2000,60,395.5000,D5,1,17467.5600,3593.9100,2,0,2,1


In [37]:
# Проверка данных
print("Информация о данных:")
print(f"Размерность: {df.shape}")
print(f"\nПропуски:")
print(df.isnull().sum().sum(), "пропусков")

# Проверка целевой переменной (Кредит_погашен)
print(f"\nРаспределение целевой переменной 'Кредит_погашен':")
print(df['Кредит_погашен'].value_counts())
print(f"\nПроцентное распределение:")
print(df['Кредит_погашен'].value_counts(normalize=True) * 100)

Информация о данных:
Размерность: (20000, 22)

Пропуски:
0 пропусков

Распределение целевой переменной 'Кредит_погашен':
Кредит_погашен
1    15998
0     4002
Name: count, dtype: int64

Процентное распределение:
Кредит_погашен
1   79.9900
0   20.0100
Name: proportion, dtype: float64


## 2. Разделение на признаки (X) и целевую переменную (y)

In [38]:
# 1. Разделение на признаки (X) и целевую переменную (y)
# Исключаем целевую переменную и признаки, которые могут создавать утечку данных

features_to_exclude = [
    'Кредит_погашен',  # целевая переменная
    'Месячный_доход',  # дублирует Годовой_доход
    'Ежемесячный_платеж'  # может создавать утечку данных
]

# Выбираем все остальные признаки
X = df.drop(columns=features_to_exclude, errors='ignore')
y = df['Кредит_погашен']

print(f"Количество признаков: {X.shape[1]}")
print(f"Количество наблюдений: {X.shape[0]}")
print(f"\nПризнаки:")
print(X.columns.tolist())
print(f"\nЦелевая переменная: Кредит_погашен")

Количество признаков: 19
Количество наблюдений: 20000

Признаки:
['Возраст', 'Пол', 'Семейное_положение', 'Уровень_образования', 'Годовой_доход', 'Статус_занятости', 'Коэффициент_долг_доход', 'Кредитный_рейтинг', 'Сумма_кредита', 'Цель_кредита', 'Процентная_ставка', 'Срок_кредита', 'Оценка_субкласс', 'Количество_открытых_счетов', 'Общий_кредитный_лимит', 'Текущий_баланс', 'История_просрочек', 'Публичные_записи', 'Количество_просрочек']

Целевая переменная: Кредит_погашен


In [39]:
# Обработка категориальных признаков
# Проверяем, какие признаки категориальные
categorical_cols = X.select_dtypes(include=['object']).columns.tolist()
numeric_cols = X.select_dtypes(include=[np.number]).columns.tolist()

print(f"Категориальные признаки ({len(categorical_cols)}): {categorical_cols}")
print(f"Числовые признаки ({len(numeric_cols)}): {numeric_cols}")

# Кодируем категориальные признаки
X_encoded = X.copy()
label_encoders = {}

for col in categorical_cols:
    le = LabelEncoder()
    X_encoded[col] = le.fit_transform(X[col].astype(str))
    label_encoders[col] = le
    print(f"\n{col}: {len(le.classes_)} уникальных значений")

print(f"\nФинальная размерность X: {X_encoded.shape}")
X_encoded.head()

Категориальные признаки (6): ['Пол', 'Семейное_положение', 'Уровень_образования', 'Статус_занятости', 'Цель_кредита', 'Оценка_субкласс']
Числовые признаки (13): ['Возраст', 'Годовой_доход', 'Коэффициент_долг_доход', 'Кредитный_рейтинг', 'Сумма_кредита', 'Процентная_ставка', 'Срок_кредита', 'Количество_открытых_счетов', 'Общий_кредитный_лимит', 'Текущий_баланс', 'История_просрочек', 'Публичные_записи', 'Количество_просрочек']

Пол: 3 уникальных значений

Семейное_положение: 4 уникальных значений

Уровень_образования: 5 уникальных значений

Статус_занятости: 5 уникальных значений

Цель_кредита: 8 уникальных значений

Оценка_субкласс: 30 уникальных значений

Финальная размерность X: (20000, 19)


,Возраст,Пол,Семейное_положение,Уровень_образования,Годовой_доход,Статус_занятости,Коэффициент_долг_доход,Кредитный_рейтинг,Сумма_кредита,Цель_кредита,Процентная_ставка,Срок_кредита,Оценка_субкласс,Количество_открытых_счетов,Общий_кредитный_лимит,Текущий_баланс,История_просрочек,Публичные_записи,Количество_просрочек
0,59,1,1,2,24240.1900,0,0.0740,743,17173.7200,1,13.3900,36,9,7,40833.4700,24302.0700,1,0,1
1,72,0,1,0,20172.9800,0,0.2190,531,22663.8900,2,17.8100,60,25,5,27968.0100,10803.0100,1,0,3
2,49,0,2,1,26181.8000,0,0.2340,779,3631.3600,0,9.5300,60,8,2,15502.2500,4505.4400,0,0,0
3,35,0,2,1,11873.8400,0,0.2640,809,14939.2300,6,7.9900,36,4,7,18157.7900,5525.6300,4,0,5
4,63,2,2,3,25326.4400,0,0.2600,663,16551.7100,1,15.2000,60,19,1,17467.5600,3593.9100,2,0,2


## 3. Разделение на обучающую и тестовую выборки

In [40]:
# 2. Разделение на обучающую и тестовую выборки
# Используем train_test_split с параметрами: test_size=0.2 и random_state=42
X_train, X_test, y_train, y_test = train_test_split(
    X_encoded, y,
    test_size=0.2,
    random_state=42,
    stratify=y  # сохраняем пропорции классов
)

print(f"Обучающая выборка: {X_train.shape[0]} наблюдений")
print(f"Тестовая выборка: {X_test.shape[0]} наблюдений")
print(f"\nРаспределение классов в обучающей выборке:")
print(y_train.value_counts())
print(f"\nРаспределение классов в тестовой выборке:")
print(y_test.value_counts())

Обучающая выборка: 16000 наблюдений
Тестовая выборка: 4000 наблюдений

Распределение классов в обучающей выборке:
Кредит_погашен
1    12798
0     3202
Name: count, dtype: int64

Распределение классов в тестовой выборке:
Кредит_погашен
1    3200
0     800
Name: count, dtype: int64


In [41]:
# Масштабирование признаков
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

X_train_scaled = pd.DataFrame(X_train_scaled, columns=X_train.columns, index=X_train.index)
X_test_scaled = pd.DataFrame(X_test_scaled, columns=X_test.columns, index=X_test.index)

print("Признаки масштабированы (StandardScaler)")
print(f"Средние значения признаков (должны быть близки к 0):")
print(X_train_scaled.mean().head())
print(f"\nСтандартные отклонения (должны быть близки к 1):")
print(X_train_scaled.std().head())

Признаки масштабированы (StandardScaler)
Средние значения признаков (должны быть близки к 0):
Возраст                0.0000
Пол                    0.0000
Семейное_положение     0.0000
Уровень_образования   -0.0000
Годовой_доход          0.0000
dtype: float64

Стандартные отклонения (должны быть близки к 1):
Возраст               1.0000
Пол                   1.0000
Семейное_положение    1.0000
Уровень_образования   1.0000
Годовой_доход         1.0000
dtype: float64


## 3. Обучение двух простых моделей

### 3.1. Логистическая регрессия

In [52]:
# Обучение модели логистической регрессии
log_reg = LogisticRegression(random_state=42, max_iter=1000)
log_reg.fit(X_train_scaled, y_train)

# Предсказания
y_train_pred_log = log_reg.predict(X_train_scaled)
y_test_pred_log = log_reg.predict(X_test_scaled)

# Оценка качества: accuracy, MAE и R²
train_accuracy_log = accuracy_score(y_train, y_train_pred_log)
test_accuracy_log = accuracy_score(y_test, y_test_pred_log)

# MAE и R² для логистической регрессии
train_mae_log = mean_absolute_error(y_train, y_train_pred_log)
test_mae_log = mean_absolute_error(y_test, y_test_pred_log)
train_r2_log = r2_score(y_train, y_train_pred_log)
test_r2_log = r2_score(y_test, y_test_pred_log)

print("Логистическая регрессия:")
print(f"Accuracy на обучающей выборке: {train_accuracy_log:.4f}")
print(f"Accuracy на тестовой выборке: {test_accuracy_log:.4f}")
print(f"\nMAE на обучающей выборке: {train_mae_log:.4f}")
print(f"MAE на тестовой выборке: {test_mae_log:.4f}")
print(f"R² на обучающей выборке: {train_r2_log:.4f}")
print(f"R² на тестовой выборке: {test_r2_log:.4f}")

# Коэффициенты модели (важность признаков)
feature_importance_log = pd.DataFrame({
    'Признак': X_train_scaled.columns,
    'Коэффициент': log_reg.coef_[0]
})
feature_importance_log['Абсолютное_значение'] = abs(feature_importance_log['Коэффициент'])
feature_importance_log = feature_importance_log.sort_values('Абсолютное_значение', ascending=False)

print(f"\nТоп-10 наиболее важных признаков (по абсолютному значению коэффициента):")
print(feature_importance_log.head(10))

Логистическая регрессия:
Accuracy на обучающей выборке: 0.8866
Accuracy на тестовой выборке: 0.8880

MAE на обучающей выборке: 0.1134
MAE на тестовой выборке: 0.1120
R² на обучающей выборке: 0.2917
R² на тестовой выборке: 0.3000

Топ-10 наиболее важных признаков (по абсолютному значению коэффициента):
                   Признак  Коэффициент  Абсолютное_значение
5         Статус_занятости      -1.3121               1.3121
7        Кредитный_рейтинг       0.9003               0.9003
6   Коэффициент_долг_доход      -0.7416               0.7416
12         Оценка_субкласс       0.1285               0.1285
4            Годовой_доход       0.0800               0.0800
14   Общий_кредитный_лимит      -0.0617               0.0617
18    Количество_просрочек       0.0409               0.0409
16       История_просрочек      -0.0320               0.0320
3      Уровень_образования       0.0269               0.0269
2       Семейное_положение      -0.0209               0.0209


### 3.2. K-ближайших соседей (KNN)

In [43]:
# Обучение модели K-ближайших соседей
# Выбираем k=5
knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X_train_scaled, y_train)

# Предсказания
y_train_pred_knn = knn.predict(X_train_scaled)
y_test_pred_knn = knn.predict(X_test_scaled)

# Оценка качества: accuracy
train_accuracy_knn = accuracy_score(y_train, y_train_pred_knn)
test_accuracy_knn = accuracy_score(y_test, y_test_pred_knn)

print("K-ближайших соседей (K=5):")
print(f"Accuracy на обучающей выборке: {train_accuracy_knn:.4f}")
print(f"Accuracy на тестовой выборке: {test_accuracy_knn:.4f}")

K-ближайших соседей (K=5):
Accuracy на обучающей выборке: 0.8949
Accuracy на тестовой выборке: 0.8608


## 4. Оценка качества и сравнение моделей

In [55]:
# Создаем таблицу сравнения моделей
comparison = pd.DataFrame({
    'Модель': ['Логистическая регрессия', 'K-ближайших соседей'],
    'Accuracy (train)': [train_accuracy_log, train_accuracy_knn],
    'Accuracy (test)': [test_accuracy_log, test_accuracy_knn]
})

print("Сравнение моделей:")
print(comparison)

print("\nСРАВНЕНИЕ МОДЕЛЕЙ:")

# Сравнение по accuracy (больше - лучше)
if test_accuracy_log > test_accuracy_knn:
    improvement = ((test_accuracy_log - test_accuracy_knn) / test_accuracy_knn) * 100
    print(f"\nЛогистическая регрессия показала лучший результат на тестовой выборке")
    print(f"  Accuracy: {test_accuracy_log:.4f} vs {test_accuracy_knn:.4f}")
    print(f"  Улучшение: {improvement:.2f}%")
    print(f"\nВывод: Логистическая регрессия лучше справляется с проверкой гипотезы.")
elif test_accuracy_knn > test_accuracy_log:
    improvement = ((test_accuracy_knn - test_accuracy_log) / test_accuracy_log) * 100
    print(f"\nK-ближайших соседей показал лучший результат на тестовой выборке")
    print(f"  Accuracy: {test_accuracy_knn:.4f} vs {test_accuracy_log:.4f}")
    print(f"  Улучшение: {improvement:.2f}%")
    print(f"\nВывод: K-ближайших соседей лучше справляется с проверкой гипотезы.")
else:
    print("\nМодели показали одинаковый результат!")
    print("Вывод: Обе модели одинаково хорошо справляются с проверкой гипотезы.")

Сравнение моделей:
                    Модель  Accuracy (train)  Accuracy (test)
0  Логистическая регрессия            0.8866           0.8880
1      K-ближайших соседей            0.8949           0.8608

СРАВНЕНИЕ МОДЕЛЕЙ:

Логистическая регрессия показала лучший результат на тестовой выборке
  Accuracy: 0.8880 vs 0.8608
  Улучшение: 3.17%

Вывод: Логистическая регрессия лучше справляется с проверкой гипотезы.


In [56]:
# Детальный отчет по классификации для обеих моделей
print("ДЕТАЛЬНЫЙ ОТЧЕТ: Логистическая регрессия")
print(classification_report(y_test, y_test_pred_log, target_names=['Не погашен', 'Погашен']))

print("\nДЕТАЛЬНЫЙ ОТЧЕТ: K-ближайших соседей")
print(classification_report(y_test, y_test_pred_knn, target_names=['Не погашен', 'Погашен']))

ДЕТАЛЬНЫЙ ОТЧЕТ: Логистическая регрессия
              precision    recall  f1-score   support

  Не погашен       0.82      0.57      0.67       800
     Погашен       0.90      0.97      0.93      3200

    accuracy                           0.89      4000
   macro avg       0.86      0.77      0.80      4000
weighted avg       0.88      0.89      0.88      4000


ДЕТАЛЬНЫЙ ОТЧЕТ: K-ближайших соседей
              precision    recall  f1-score   support

  Не погашен       0.77      0.44      0.56       800
     Погашен       0.87      0.97      0.92      3200

    accuracy                           0.86      4000
   macro avg       0.82      0.70      0.74      4000
weighted avg       0.85      0.86      0.85      4000



## 5. Визуализация результатов

### 5.1. Матрица ошибок (Confusion Matrix)


In [ ]:
# Матрица ошибок для обеих моделей
cm_log = confusion_matrix(y_test, y_test_pred_log)
cm_knn = confusion_matrix(y_test, y_test_pred_knn)

# Создаем визуализацию матриц ошибок
fig = make_subplots(
    rows=1, cols=2,
    subplot_titles=('Логистическая регрессия', 'K-ближайших соседей'),
    specs=[[{'type': 'heatmap'}, {'type': 'heatmap'}]]
)

# Матрица ошибок для логистической регрессии
fig.add_trace(
    go.Heatmap(
        z=cm_log,
        x=['Не погашен', 'Погашен'],
        y=['Не погашен', 'Погашен'],
        colorscale='Blues',
        text=cm_log,
        texttemplate='%{text}',
        textfont={"size": 14},
        showscale=True,
        colorbar=dict(x=0.45, len=0.4)
    ),
    row=1, col=1
)

# Матрица ошибок для KNN
fig.add_trace(
    go.Heatmap(
        z=cm_knn,
        x=['Не погашен', 'Погашен'],
        y=['Не погашен', 'Погашен'],
        colorscale='Greens',
        text=cm_knn,
        texttemplate='%{text}',
        textfont={"size": 14},
        showscale=True,
        colorbar=dict(x=1.02, len=0.4)
    ),
    row=1, col=2
)

# Добавляем аннотации с метриками
fig.add_annotation(
    text=f"Accuracy: {test_accuracy_log:.4f}",
    xref="x1", yref="y1",
    x=0.5, y=-0.15,
    xanchor='center', yanchor='top',
    showarrow=False,
    font=dict(size=12, color='black')
)

fig.add_annotation(
    text=f"Accuracy: {test_accuracy_knn:.4f}",
    xref="x2", yref="y2",
    x=0.5, y=-0.15,
    xanchor='center', yanchor='top',
    showarrow=False,
    font=dict(size=12, color='black')
)

fig.update_layout(
    height=500,
    title_text="Матрица ошибок (Confusion Matrix) - Тестовая выборка",
    title_x=0.5
)

fig.update_xaxes(title_text="Предсказанный класс", row=1, col=1)
fig.update_yaxes(title_text="Реальный класс", row=1, col=1)
fig.update_xaxes(title_text="Предсказанный класс", row=1, col=2)
fig.update_yaxes(title_text="Реальный класс", row=1, col=2)

fig.show()

# Выводим числовые значения матриц
print("\nМатрица ошибок - Логистическая регрессия:")
print("                Предсказано")
print("              Не погашен  Погашен")
print(f"Реально Не погашен    {cm_log[0,0]:5d}    {cm_log[0,1]:5d}")
print(f"        Погашен       {cm_log[1,0]:5d}    {cm_log[1,1]:5d}")

print("\nМатрица ошибок - K-ближайших соседей:")
print("                Предсказано")
print("              Не погашен  Погашен")
print(f"Реально Не погашен    {cm_knn[0,0]:5d}    {cm_knn[0,1]:5d}")
print(f"        Погашен       {cm_knn[1,0]:5d}    {cm_knn[1,1]:5d}")


Матрица ошибок - Логистическая регрессия:
                Предсказано
              Не погашен  Погашен
Реально Не погашен      454      346
        Погашен         102     3098

Матрица ошибок - K-ближайших соседей:
                Предсказано
              Не погашен  Погашен
Реально Не погашен      350      450
        Погашен         107     3093


### 5.2. Визуализация важности признаков


In [ ]:
# Визуализация топ-10 наиболее важных признаков (для логистической регрессии)
top_features = feature_importance_log.head(10)

fig = go.Figure()

# Создаем цветовую схему: положительные коэффициенты - синие, отрицательные - красные
colors = ['red' if x < 0 else 'blue' for x in top_features['Коэффициент']]

fig.add_trace(go.Bar(
    x=top_features['Абсолютное_значение'],
    y=top_features['Признак'],
    orientation='h',
    marker=dict(color=colors),
    text=[f"{x:.4f}" for x in top_features['Коэффициент']],
    textposition='outside',
    name='Коэффициент'
))

fig.update_layout(
    title='Топ-10 наиболее важных признаков (Логистическая регрессия)',
    xaxis_title='Абсолютное значение коэффициента',
    yaxis_title='Признак',
    height=500,
    showlegend=False
)

# Добавляем аннотацию
fig.add_annotation(
    text="Синий = увеличивает вероятность погашения<br>Красный = уменьшает вероятность погашения",
    xref="paper", yref="paper",
    x=0.98, y=0.02,
    xanchor='right', yanchor='bottom',
    showarrow=False,
    bgcolor="rgba(255,255,255,0.8)",
    bordercolor="black",
    borderwidth=1,
    font=dict(size=10)
)

fig.show()

## 6. Выводы

### 6.1. Подтверждение гипотезы


In [57]:
print("АНАЛИЗ ПОДТВЕРЖДЕНИЯ ГИПОТЕЗЫ")

print("\nГипотеза: Можно ли предсказать, будет ли кредит погашен, на основе характеристик заемщика?")

print(f"\nРезультаты моделирования:")
print(f"- Accuracy (логистическая регрессия): {test_accuracy_log:.4f}")
print(f"- Accuracy (K-ближайших соседей): {test_accuracy_knn:.4f}")

# Определяем лучшую модель
best_accuracy = max(test_accuracy_log, test_accuracy_knn)

if best_accuracy > 0.8:
    print("\nГИПОТЕЗА ПОДТВЕРЖДЕНА!")
    print("  Модели показывают высокую способность предсказывать погашение кредита.")
    print(f"  Accuracy > 0.8 означает, что модель правильно классифицирует более 80% случаев.")
elif best_accuracy > 0.7:
    print("\nГИПОТЕЗА ЧАСТИЧНО ПОДТВЕРЖДЕНА")
    print("  Модели показывают умеренную способность предсказывать погашение кредита.")
    print(f"  Accuracy > 0.7 означает, что модель правильно классифицирует более 70% случаев.")
else:
    print("\nГИПОТЕЗА НЕ ПОДТВЕРЖДЕНА")
    print("  Модели показывают низкую способность предсказывать погашение кредита.")
    print("  Возможно, необходимы дополнительные признаки или более сложные модели.")

print(f"\nЛучшая модель показывает accuracy: {best_accuracy:.4f} ({best_accuracy*100:.2f}%)")


АНАЛИЗ ПОДТВЕРЖДЕНИЯ ГИПОТЕЗЫ

Гипотеза: Можно ли предсказать, будет ли кредит погашен, на основе характеристик заемщика?

Результаты моделирования:
- Accuracy (логистическая регрессия): 0.8880
- Accuracy (K-ближайших соседей): 0.8608

ГИПОТЕЗА ПОДТВЕРЖДЕНА!
  Модели показывают высокую способность предсказывать погашение кредита.
  Accuracy > 0.8 означает, что модель правильно классифицирует более 80% случаев.

Лучшая модель показывает accuracy: 0.8880 (88.80%)


### 6.2. Анализ важности признаков


In [59]:
print("АНАЛИЗ ВАЖНОСТИ ПРИЗНАКОВ")

print("\nТоп-10 наиболее важных признаков для предсказания погашения кредита:")
for idx, row in feature_importance_log.head(10).iterrows():
    direction = "увеличивает вероятность" if row['Коэффициент'] > 0 else "уменьшает вероятность"
    print(f"{idx+1:2d}. {row['Признак']:30s} | Коэф: {row['Коэффициент']:8.4f} | {direction}")


print("\nИНТЕРПРЕТАЦИЯ:")
print("\n1. Положительный коэффициент означает, что увеличение признака увеличивает вероятность погашения кредита")
print("2. Отрицательный коэффициент означает, что увеличение признака уменьшает вероятность погашения кредита")
print("3. Чем больше абсолютное значение коэффициента, тем сильнее влияние признака")

print("\nКЛЮЧЕВЫЕ ВЫВОДЫ:")
top_5 = feature_importance_log.head(5)
print("\nНаиболее важные факторы, влияющие на погашение кредита:")
for idx, row in top_5.iterrows():
    impact = "увеличивает" if row['Коэффициент'] > 0 else "уменьшает"
    print(f"  • {row['Признак']} - {impact} вероятность погашения")

# Проверяем гипотезы о влиянии признаков
print("\nПРОВЕРКА ГИПОТЕЗ О ВЛИЯНИИ ПРИЗНАКОВ:")


# Кредитный рейтинг (ожидаем положительный коэффициент)
credit_score_coef = feature_importance_log[feature_importance_log['Признак'] == 'Кредитный_рейтинг']
if len(credit_score_coef) > 0:
    coef_val = credit_score_coef.iloc[0]['Коэффициент']
    if coef_val > 0:
        print("Кредитный рейтинг: подтверждено - чем выше рейтинг, тем выше вероятность погашения")
    else:
        print("Кредитный рейтинг: не подтверждено - неожиданный знак коэффициента")

# Годовой доход (ожидаем положительный коэффициент)
income_coef = feature_importance_log[feature_importance_log['Признак'] == 'Годовой_доход']
if len(income_coef) > 0:
    coef_val = income_coef.iloc[0]['Коэффициент']
    if coef_val > 0:
        print("Годовой доход: подтверждено - чем выше доход, тем выше вероятность погашения")
    else:
        print("Годовой доход: не подтверждено - неожиданный знак коэффициента")

# Коэффициент долг/доход (ожидаем отрицательный коэффициент)
dti_coef = feature_importance_log[feature_importance_log['Признак'] == 'Коэффициент_долг_доход']
if len(dti_coef) > 0:
    coef_val = dti_coef.iloc[0]['Коэффициент']
    if coef_val < 0:
        print("Коэффициент долг/доход: подтверждено - чем выше коэффициент, тем ниже вероятность погашения")
    else:
        print("Коэффициент долг/доход: не подтверждено - неожиданный знак коэффициента")

# История просрочек (ожидаем отрицательный коэффициент)
delinq_coef = feature_importance_log[feature_importance_log['Признак'] == 'История_просрочек']
if len(delinq_coef) > 0:
    coef_val = delinq_coef.iloc[0]['Коэффициент']
    if coef_val < 0:
        print("История просрочек: подтверждено - чем больше просрочек, тем ниже вероятность погашения")
    else:
        print("История просрочек: не подтверждено - неожиданный знак коэффициента")

АНАЛИЗ ВАЖНОСТИ ПРИЗНАКОВ

Топ-10 наиболее важных признаков для предсказания погашения кредита:
 6. Статус_занятости               | Коэф:  -1.3121 | уменьшает вероятность
 8. Кредитный_рейтинг              | Коэф:   0.9003 | увеличивает вероятность
 7. Коэффициент_долг_доход         | Коэф:  -0.7416 | уменьшает вероятность
13. Оценка_субкласс                | Коэф:   0.1285 | увеличивает вероятность
 5. Годовой_доход                  | Коэф:   0.0800 | увеличивает вероятность
15. Общий_кредитный_лимит          | Коэф:  -0.0617 | уменьшает вероятность
19. Количество_просрочек           | Коэф:   0.0409 | увеличивает вероятность
17. История_просрочек              | Коэф:  -0.0320 | уменьшает вероятность
 4. Уровень_образования            | Коэф:   0.0269 | увеличивает вероятность
 3. Семейное_положение             | Коэф:  -0.0209 | уменьшает вероятность

ИНТЕРПРЕТАЦИЯ:

1. Положительный коэффициент означает, что увеличение признака увеличивает вероятность погашения кредита
2. Отрицател

### 6.3. Итоговые выводы


ИТОГОВЫЕ ВЫВОДЫ

1. СРАВНЕНИЕ МОДЕЛЕЙ:
   - Логистическая регрессия показала лучший результат по метрике accuracy
   - Рекомендуется использовать логистическую регрессию для предсказания погашения кредита

2. КАЧЕСТВО ПРЕДСКАЗАНИЙ:
   - Лучший accuracy: 0.8880 (88.80%)
   - Модель правильно классифицирует 88.8% случаев

3. ВАЖНОСТЬ ПРИЗНАКОВ:
   Наиболее важные факторы для предсказания погашения кредита:
   - Статус_занятости
   - Кредитный_рейтинг
   - Коэффициент_долг_доход
   - Оценка_субкласс
   - Годовой_доход

4. ПРАКТИЧЕСКОЕ ПРИМЕНЕНИЕ:
   - Модель может быть использована банками для оценки риска непогашения кредита
   - На основе модели можно определить, какие факторы наиболее критичны для заемщика
   - Заемщики могут понять, какие характеристики улучшить для повышения вероятности одобрения кредита

5. ОГРАНИЧЕНИЯ И РЕКОМЕНДАЦИИ:
   - Модель основана на исторических данных и может не учитывать все факторы
   - Для улучшения качества можно:
     - Добавить больше признаков (например, регион, тип недвижимости)
     - Попробовать более сложные модели (Random Forest, Gradient Boosting)
     - Учесть временные тренды и экономические факторы
     - Применить feature engineering для создания новых признаков